# Tools in LangChain

In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.8 MB 1.9 MB/s eta 0:00:02
   ----------- ---------------------------- 1.0/3.8 MB 1.9 MB/s eta 0:00:02
   ------------------- -------------------- 1.8/3.8 MB 2.3 MB/s eta 0:00:01
   --------------------------- ------------ 2.6/3.8 MB 2.5 MB/s eta 0:00:01
   --------------------------------- ------ 3.1/3.8 MB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.1 MB 8.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.3/3.1 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.1 MB 2.9 MB/s eta 0:00:01
   -------------------------- ------------- 2.1/3.1 MB 3.0 MB/s eta 0:00:01
   ------------------------------

### Built-in Tool - DuckDuckGo Search

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

Stay here for real-time updates on breaking news from India and across the world that you can't miss: Air India announces flight cancellations at 8 locations for today 12:50 (IST) May 13 India ... India-Pakistan Ceasefire News LIVE: India declares Pakistani official, working at Pak high commission in Delhi, persona non grata ... Charge d' Affaires, Pakistan High Commission, was issued a demarche to this effect today. May 13, 2025, 20:07:46 (IST) ... A top police officer said that a massive cordon and search operation was launched in ... India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Get updated with current top news stories from India and the world only on Zee News. Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusi

In [5]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Built-in Tool - Shell Tool

In [6]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
'ls' is not recognized as an internal or external command,
operable program or batch file.



c:\Users\mange\OneDrive\Desktop\langChain_Models\.venv\lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

In [7]:
from langchain_core.tools import tool

In [8]:
# Step 1 - create a function

def multiply(a,b):
    """Multiply two numbers"""
    return a*b

In [9]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [10]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [11]:
result = multiply.invoke({"a":3, "b":5})

In [12]:
print(result)

15


In [13]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [14]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method 2 - Using StructuredTool

In [15]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [16]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [17]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [18]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [19]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Method 3 - Using BaseTool Class

In [20]:
from langchain.tools import BaseTool
from typing import Type

In [21]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [28]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [29]:
multiply_tool = MultiplyTool()

In [30]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Toolkit

In [31]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [32]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [33]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
